In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14595,2024-12-25,Eua Nba,14:00,New York Knicks,San Antonio Spurs,1.27,4.05,221.5,1.83,2.02,-10.5,1.99,1.85,dQK84Dk5,0.787402,0.246914,0.546448,0.495050,0.034315,0.8,0.8,NaN,NaN,180.994,68.307210,0.377400,233.112,47.095971,0.202032,144.748,275.896,154.29,298.92,0.0,0.0,0.0,0.0,0.739006,0.069792,0.051560,-0.30,-0.060,-4.500000,0.774566,0.7,-0.074566,1.150000e+00,2.300000e-01,1.326087e+01,0.383755,0.4,0.016245
14596,2024-12-25,Eua Nba,16:30,Dallas Mavericks,Minnesota Timberwolves,1.51,2.68,219.5,1.82,2.03,-5.5,2.00,1.84,rsJG2iKH,0.662252,0.373134,0.549451,0.492611,0.035386,0.6,0.2,NaN,NaN,169.900,37.718061,0.222002,164.176,23.233917,0.141518,166.442,177.414,159.72,163.28,0.0,0.0,0.0,0.0,0.394900,0.077139,0.058926,-1.01,-0.202,-2.524752,0.744458,0.6,-0.144458,8.881784e-16,1.776357e-16,9.457559e+15,0.642187,0.4,-0.242187
14597,2024-12-25,Eua Nba,19:00,Boston Celtics,Philadelphia 76ers,1.27,4.05,220.5,1.87,1.97,-9.5,1.99,1.85,v9CP0VlU,0.787402,0.246914,0.534759,0.507614,0.034315,0.4,0.6,NaN,NaN,135.934,20.873904,0.153559,193.010,35.309529,0.182941,139.442,198.534,118.80,199.65,0.0,0.0,0.0,0.0,0.739006,0.036828,0.051560,-1.61,-0.322,-0.838509,0.829730,0.7,-0.129730,2.200000e+00,4.400000e-01,6.931818e+00,0.507558,0.5,-0.007558
14598,2024-12-25,Eua Nba,22:00,Golden State Warriors,Los Angeles Lakers,1.59,2.46,222.5,1.85,1.99,-4.5,2.02,1.83,fLnZKYZo,0.628931,0.406504,0.540541,0.502513,0.035435,0.4,0.2,NaN,NaN,215.836,60.321874,0.279480,242.428,86.823751,0.358142,221.380,235.280,154.35,318.66,0.0,0.0,0.0,0.0,0.303794,0.051560,0.069792,-0.67,-0.134,-4.402985,0.577143,0.5,-0.077143,8.900000e-01,1.780000e-01,8.202247e+00,0.529507,0.4,-0.129507
14599,2024-12-25,Austrália Nbl,04:00,Tasmania JackJumpers,NZ Breakers,1.26,3.90,174.5,1.86,1.80,-9.5,1.94,1.77,t0e9kHXr,0.793651,0.256410,0.537634,0.555556,0.050061,0.2,1.0,NaN,NaN,203.560,84.640578,0.415802,291.782,66.680008,0.228527,150.410,309.470,130.68,313.02,0.0,0.0,0.0,0.0,0.723551,0.023184,0.064802,-1.87,-0.374,-0.695187,0.609854,0.6,-0.009854,2.420000e+00,4.840000e-01,5.991736e+00,0.393552,0.3,-0.093552
14600,2024-12-25,Ásia Easl,07:40,Ryukyu,New Taipei Kings,1.27,3.26,177.5,1.88,1.78,-9.5,1.98,1.70,C8yHsDcS,0.787402,0.306748,0.531915,0.561798,0.094150,0.0,0.0,NaN,NaN,134.344,41.250128,0.307049,132.386,35.535890,0.268426,0.000,0.000,111.02,184.68,0.0,0.0,0.0,0.0,0.621255,0.038640,0.107603,0.00,0.000,inf,0.000000,0.0,0.000000,0.000000e+00,0.000000e+00,inf,0.000000,0.0,0.000000
14601,2024-12-25,Ásia Easl,09:10,Taoyuan Pauian Pilots,Hiroshima D.,1.34,2.87,165.5,1.82,1.84,-8.5,2.03,1.67,6VMPABRE,0.746269,0.348432,0.549451,0.543478,0.094701,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,116.00,102.96,0.0,0.0,0.0,0.0,0.513954,0.007728,0.137599,0.00,0.000,inf,0.000000,0.0,0.000000,0.000000e+00,0.000000e+00,inf,0.000000,0.0,0.000000
14602,2024-12-25,Bahrein Premier League,14:00,Al-Hala,Al Najma,1.44,2.52,163.5,1.81,1.85,-6.5,2.05,1.65,IBFSMtbK,0.694444,0.396825,0.552486,0.540541,0.091270,0.2,0.0,NaN,NaN,174.572,75.533381,0.432678,271.454,155.261781,0.571964,160.910,0.000,227.92,458.64,0.0,0.0,0.0,0.0,0.385695,0.015456,0.152888,-0.67,-0.134,-3.283582,0.000000,0.0,0.000000,0.000000e+00,0.000000e+00,inf,0.000000,0.0,0.000000
14603,2024-12-25,Catar Liga De Basquete Do Catar,11:00,Al Shamal,Al Ahli,2.62,1.41,157.5,1.85,1.81,4.5,2.03,1.67,rLTyluR8,0.381679,0.709220,0.540541,0.552486,0.090899,0.4,0.6,NaN,NaN,210.266,90.084226,0.428

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Eua Nba,New York Knicks,San Antonio Spurs,221.5,1.83,0.9222,Over
1,16:30,Eua Nba,Dallas Mavericks,Minnesota Timberwolves,219.5,1.82,0.8833,Over
2,19:00,Eua Nba,Boston Celtics,Philadelphia 76ers,220.5,1.87,0.8964,Over
3,22:00,Eua Nba,Golden State Warriors,Los Angeles Lakers,222.5,1.85,0.8994,Over
